In [2]:
# Load necessary packages

import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

# from google.colab import files

%matplotlib inline

In [3]:
# uploaded = files.upload()

In [4]:
# Read data and present

train = pd.read_csv('trainingData.csv')
valid = pd.read_csv('validationData.csv')
train.head()

,deck,nofGames,nOfPlayers,winRate
0,archers;arrows;baby-dragon;balloon;bats;fireba...,44,2,0.58511
1,archers;arrows;baby-dragon;balloon;bomber;free...,143,3,0.53767
2,archers;arrows;baby-dragon;balloon;bomber;gian...,61,3,0.34375
3,archers;arrows;baby-dragon;balloon;cannon;free...,162,1,0.49394
4,archers;arrows;baby-dragon;balloon;electro-wiz...,57,1,0.55833


In [5]:
valid.head()

,deck,nofGames,nOfPlayers,winRate
0,archers;arrows;baby-dragon;bandit;elixir-colle...,130,32,0.48496
1,archers;arrows;baby-dragon;elixir-collector;go...,495,75,0.47289
2,archers;arrows;baby-dragon;golem;lightning;meg...,271,25,0.52372
3,archers;arrows;baby-dragon;golem;lightning;meg...,125,20,0.58594
4,archers;arrows;balloon;barbarians;fireball;gia...,130,23,0.50000


In [6]:
# Helper functions to preprocess data to bag-of-cards format

def unnest(df, col):
    unnested = (df.apply(lambda x: pd.Series(x[col]), axis=1)
                .stack()
                .reset_index(level=1, drop=True))
    unnested.name = col
    return df.drop(col, axis=1).join(unnested)

def to_bag_of_cards(df):
    df['ind'] = np.arange(df.shape[0]) + 1
    df_orig = df.copy()
    df['deck'] = df['deck'].apply(lambda d: d.split(';'))
    df = unnest(df, 'deck')
    df['value'] = 1
    df_bag = df.pivot(index='ind', columns='deck', values='value')
    df_bag[df_bag.isna()] = 0
    df_bag = df_bag.astype('int')
    return pd.concat([df_orig.set_index('ind'), df_bag], axis=1)

In [7]:
train = to_bag_of_cards(train)
valid = to_bag_of_cards(valid)
train.head()
train

,deck,nofGames,nOfPlayers,winRate,archers,arrows,baby-dragon,balloon,bandit,barbarian-barrel,...,the-log,three-musketeers,tombstone,tornado,valkyrie,witch,wizard,x-bow,zap,zappies
ind,,,,,,,,,,,,,,,,,,,,,
1,archers;arrows;baby-dragon;balloon;bats;fireba...,44,2,0.58511,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,archers;arrows;baby-dragon;balloon;bomber;free...,143,3,0.53767,1,1,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,archers;arrows;baby-dragon;balloon;bomber;gian...,61,3,0.34375,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,archers;arrows;baby-dragon;balloon;cannon;free...,162,1,0.49394,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,archers;arrows;baby-dragon;balloon;electro-wiz...,57,1,0.55833,1,1,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6,archers;arrows;baby-dragon;balloon;elite-barba...,68,1,0.47183,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,archers;arrows;baby-dragon;balloon;fire-spirit...,42,2,0.47778,1,1,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
8,archers;arrows;baby-dragon;balloon;fireball;la...,27,1,0.51667,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,archers;arrows;baby-dragon;balloon;freeze;ice-...,303,1,0.57353,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
valid.head()

,deck,nofGames,nOfPlayers,winRate,archers,arrows,baby-dragon,balloon,bandit,barbarian-barrel,...,the-log,three-musketeers,tombstone,tornado,valkyrie,witch,wizard,x-bow,zap,zappies
ind,,,,,,,,,,,,,,,,,,,,,
1,archers;arrows;baby-dragon;bandit;elixir-colle...,130,32,0.48496,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,archers;arrows;baby-dragon;elixir-collector;go...,495,75,0.47289,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,archers;arrows;baby-dragon;golem;lightning;meg...,271,25,0.52372,1,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,archers;arrows;baby-dragon;golem;lightning;meg...,125,20,0.58594,1,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,archers;arrows;balloon;barbarians;fireball;gia...,130,23,0.50000,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# Sort data by number of games played

train = train.sort_values('nofGames', ascending=False)
valid = valid.sort_values('nofGames', ascending=False)

In [10]:
# Specify example model fitting function and R squared metric

from sklearn.svm import SVR

def R2(x, y):
    return 1 - np.sum(np.square(x - y)) / np.sum(np.square(y - np.mean(y)))

def fit_svm(data):
    svr = SVR(kernel='rbf', gamma=1.0/90, C=1.0, epsilon=0.02, shrinking=False)
    svr.fit(data.drop(['deck', 'nofGames', 'nOfPlayers', 'winRate'], axis=1), data['winRate'])
    return svr

sizes = (np.arange(10) + 6) * 100

In [11]:
# Fit and predict on models of various training sizes

fit_list = list(map(lambda size: fit_svm(train.iloc[:size]), sizes))
pred_list = list(map(lambda fit: fit.predict(valid.drop(['deck', 'nofGames', 'nOfPlayers', 'winRate'], axis=1)),
                     fit_list))

In [12]:
from sklearn.svm import NuSVR

nuSVM = NuSVR(kernel='rbf', gamma=1.0/90, C=1.0, nu=0.01, shrinking=False)
nuSVM.fit(train.drop(['deck', 'nofGames', 'nOfPlayers', 'winRate'], axis=1), train['winRate'], sample_weight=train['nofGames']/np.max(train['nofGames']))
preds = nuSVM.predict(valid.drop(['deck', 'nofGames', 'nOfPlayers', 'winRate'], axis=1))
print(R2(preds, valid['winRate']))
print(len(nuSVM.support_))

train2 = train.loc[nuSVM.support_]
nuSVM2 = NuSVR(kernel='rbf', gamma=1.0/90, C=1.0, nu=0.01, shrinking=False)
nuSVM2.fit(train2.drop(['deck', 'nofGames', 'nOfPlayers', 'winRate'], axis=1), train2['winRate'], sample_weight=train2['nofGames']/np.max(train2['nofGames']))
preds2 = nuSVM2.predict(valid.drop(['deck', 'nofGames', 'nOfPlayers', 'winRate'], axis=1))
print(R2(preds2, valid['winRate']))
print(len(nuSVM2.support_))

KeyboardInterrupt: 

In [ ]:
np.max(train['nofGames']/np.max(train['nofGames']))


In [ ]:
# Calculate R squared scores

r2 = list(map(lambda p: R2(p, valid['winRate']), pred_list))
r2

In [ ]:
_ = plt.plot(sizes, r2)

In [ ]:
np.mean(r2)

In [ ]:
# Save hyperparameteres and selected indices in submission format

with open('example_sub_python.txt', 'a') as f:
    for size in sizes:
        ind_text = ','.join(list(map(str, train.index.values[:size])))
        text = ';'.join(['0.02', '1.0', str(1.0 / 90), ind_text])
        f.write(text + '\n')

In [13]:

wintrain = train.sort_values(["nofGames","winRate"], ascending=False)
winvalid = valid.sort_values(["nofGames","winRate"], ascending=False)

In [ ]:
wintrain["differ"]= wintrain["winRate"]

In [188]:
def differ(x,y):
    return x.ne(y).sum()

def distance(data):
    dist= pd.DataFrame(index=data.index, columns=np.arange(100))
    
    for row in range(len(data.index)):
        random= np.random.randint(100000,size=100)
        for other in range(len(random)):
        
            dist.iloc[row,other]=differ(data.iloc[row,:],data.iloc[other,:])
    
    return dist

In [49]:
train.iloc[0,4:].ne(train.iloc[1,4:]).sum()

12

In [181]:

r=np.random.randint(100000,size=10)

for i in range(len(r)):
    print(r[i])

87768
68014
20665
62538
78856
15902
35036
28978
92217
32379


In [198]:
sort=train.sort_values('nofGames', ascending=False).drop(['deck', 'nofGames', 'nOfPlayers', 'winRate'], axis=1).head(10)

sort

,archers,arrows,baby-dragon,balloon,bandit,barbarian-barrel,barbarian-hut,barbarians,bats,battle-ram,...,the-log,three-musketeers,tombstone,tornado,valkyrie,witch,wizard,x-bow,zap,zappies
ind,,,,,,,,,,,,,,,,,,,,,
67117,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
96911,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
53262,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,1,0
62986,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
44606,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
96972,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9101,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
41397,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
15151,0,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [196]:
distances=distance(sort)


IndexError: single positional indexer is out-of-bounds

,nofGames,nOfPlayers,winRate,archers,arrows,baby-dragon,balloon,bandit,barbarian-barrel,barbarian-hut,...,the-log,three-musketeers,tombstone,tornado,valkyrie,witch,wizard,x-bow,zap,zappies
count,5.000,5.000000,5.000000,5.0,5.0,5.0,5.0,5.000000,5.0,5.0,...,5.000000,5.000000,5.0,5.0,5.0,5.0,5.0,5.0,5.000000,5.0
mean,586182.800,80298.200000,0.511464,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,...,0.400000,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.600000,0.0
std,234760.753,32012.152422,0.016269,0.0,0.0,0.0,0.0,0.447214,0.0,0.0,...,0.547723,0.547723,0.0,0.0,0.0,0.0,0.0,0.0,0.547723,0.0
min,286105.000,42580.000000,0.493050,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25%,411205.000,59331.000000,0.494710,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
50%,654996.000,73773.000000,0.520190,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0
75%,710137.000,107071.000000,0.522110,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0
max,868471.000,118736.000000,0.527260,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,...,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0


In [ ]:
distances.to_csv()
sortowane.to_csv()

In [142]:
train.drop(['deck', 'nofGames', 'nOfPlayers', 'winRate'], axis=1).head()

,archers,arrows,baby-dragon,balloon,bandit,barbarian-barrel,barbarian-hut,barbarians,bats,battle-ram,...,the-log,three-musketeers,tombstone,tornado,valkyrie,witch,wizard,x-bow,zap,zappies
ind,,,,,,,,,,,,,,,,,,,,,
67117,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
96911,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
53262,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,1,0
62986,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
44606,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [83]:
type(train.iloc[:10,4:])

pandas.core.frame.DataFrame

In [152]:
df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),columns=['a', 'b', 'c'])
df2
df2[1,2]=8
df2.iloc[1,2]=100
df2.sum(axis=1)

0     14
1    117
2     32
dtype: int64

In [111]:
dist= pd.DataFrame(index=np.arange(10), columns=np.arange(10))

dist

,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
dt=pd.DataFrame(columns=np.arange(10))

In [107]:
dt

,0,1,2,3,4,5,6,7,8,9


In [120]:
dist.iloc[1,2]=differ(train.iloc[1,4:],train.iloc[2,4:])
dist

,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
  dist= pd.DataFrame(index=data.index, columns=np.arange(len(data)))
    for row in data:
        for other in data:
            dist.iloc[row,other]=differ(data.iloc[row,4:],data.iloc[other,4:])

14

In [133]:
dane=train.iloc[:10,4:]
cnt=0
for row in range(len(dane.index)):
    print (row)
    cnt+=1
cnt    

0
1
2
3
4
5
6
7
8
9


10